In [1]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self. children = {}
        
    def inc(self, numOccur):
        self.count += numOccur
        
    def disp(self, ind=1):
        print(' '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind + 1)

### FP树构建函数

In [2]:
def createTree(dataSet, minSup=1):    # 数据集，最小适应度
    headerTable = {}
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    # 移除不满足最小支持度的元素项
    for k in list(headerTable.keys()):
        if headerTable[k] < minSup:
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    # 如果没有元素满足要求，则退出
    if len(freqItemSet) == 0:
        return None, None
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]
    retTree = treeNode('Null Set', 1, None)    # 头部指针
    for tranSet, count in dataSet.items():
        localD = {}
        # 根据全局频率对每个事务中的元素进行排序
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key = lambda p : p[1], reverse = True)]
            # 使用排序后的频率项集对树进行填充
            updateTree(orderedItems, retTree, headerTable, count)
    return retTree, headerTable

In [3]:
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:
        inTree.children[items[0]].inc(count)    # 如果存在，增加计数
    else:
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:
        # 对剩下的元素迭代调用updateTree
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

In [4]:
def updateHeader(nodeToTest, targetNode):
    while nodeToTest.nodeLink != None:
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [5]:
def loadSimplDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

In [6]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [7]:
simpDat = loadSimplDat()
initSet = createInitSet(simpDat)
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [8]:
myFPtree, myHeaderTab = createTree(initSet, 3)
myFPtree.disp()

  Null Set   1
   z   5
    r   1
    x   3
     t   3
      y   2
       s   2
      r   1
       y   1
   x   1
    r   1
     s   1


## 从FP树中抽取频繁项集的三个基本步骤：
1. 从FP树中获得条件模式基；  
2. 利用条件模式基，构建一个条件FP树；  
3. 迭代重复步骤1.步骤2.，直到树包含一个元素项为止。

### 发现以给定元素项结尾的所有路径的函数

In [9]:
def ascendTree(leafNode, predixPath):
    # 迭代上溯整棵树
    if leafNode.parent != None:
        predixPath.append(leafNode.name)    # 收集遇到的元素项名称
        ascendTree(leafNode.parent, predixPath)

In [10]:
# 遍历链表直到队尾，每遇到一个元素都调用ascendTree上溯FP树
def findPrefixPath(basePat, treeNode):
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    # 返回条件模式基字典
    return condPats   

In [11]:
findPrefixPath('x', myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [12]:
findPrefixPath('z', myHeaderTab['z'][1])

{}

In [13]:
findPrefixPath('r', myHeaderTab['r'][1])

{frozenset({'z'}): 1, frozenset({'x'}): 1, frozenset({'t', 'x', 'z'}): 1}

### 递归查找频繁项集的mineTree函数

In [14]:
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    # 对头指针表中的元素按其出现频率从小到大排序
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p : p[0])]
    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        # 从条件模式基构建条件FP树
        myCondTree, myHead = createTree(condPattBases, minSup)
        if myHead != None:
            print('conditional tree for: ',newFreqSet)
            myCondTree.disp(1)
            # 挖掘条件FP树
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [15]:
freqItems = []
mineTree(myFPtree, myHeaderTab, 3, set([]), freqItems)

conditional tree for:  {'s'}
  Null Set   1
   x   3
conditional tree for:  {'t'}
  Null Set   1
   z   3
    x   3
conditional tree for:  {'x', 't'}
  Null Set   1
   z   3
conditional tree for:  {'x'}
  Null Set   1
   z   3
conditional tree for:  {'y'}
  Null Set   1
   z   3
    x   3
     t   3
conditional tree for:  {'y', 't'}
  Null Set   1
   z   3
    x   3
conditional tree for:  {'y', 'x', 't'}
  Null Set   1
   z   3
conditional tree for:  {'y', 'x'}
  Null Set   1
   z   3


In [16]:
freqItems

[{'r'},
 {'s'},
 {'s', 'x'},
 {'t'},
 {'t', 'x'},
 {'t', 'x', 'z'},
 {'t', 'z'},
 {'x'},
 {'x', 'z'},
 {'y'},
 {'t', 'y'},
 {'t', 'x', 'y'},
 {'t', 'x', 'y', 'z'},
 {'t', 'y', 'z'},
 {'x', 'y'},
 {'x', 'y', 'z'},
 {'y', 'z'},
 {'z'}]

## 从新闻网站点击流中挖掘

In [35]:
parseDat = [line.split() for line in open('kosarak.dat').readlines()]
initSet = createInitSet(parseDat)
myFPtree, myHeaderTab = createTree(initSet, 100000)
myFPtree.disp()

  Null Set   1
   6   119877


In [36]:
myFreqList = []
mineTree(myFPtree, myHeaderTab, 100000, set([]), myFreqList)

In [37]:
len(myFreqList)

1

In [38]:
myFreqList

[{'6'}]